In [1]:
# !pip install openpyxl

In [97]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 332397/332397 [00:00<00:00, 957881.21it/s]


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15658241,Olmstead,140.686089,Germany,Female,81,48,438091.980941,2,1,0,1.0,0,0,2,GOLD,2194.0,1-2-2023
1,15627162,Blesing,614.692750,Germany,Male,27,6,155245.230616,1,1,0,1.0,0,0,1,SILVER,675.0,1-3-2020
2,15986692,Leiva,665.305005,Germany,Male,35,7,157859.682813,2,1,1,1.0,0,0,5,SILVER,758.0,1-10-2020
3,15443069,Jamieson,541.210330,France,Male,36,7,0.000000,2,0,1,0.0,1,0,5,GOLD,1365.0,1-10-2020
4,15611786,Tsui,621.240000,Spain,Female,69,9,0.000000,1,0,1,0.0,0,0,5,PLATINUM,401.0,1-1-2020


In [98]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [99]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 332397/332397 [00:00<00:00, 938036.79it/s]


In [100]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [101]:
state_mapper

{'Germany': 'Connecticut', 'France': 'Virginia', 'Spain': 'Vermont'}

In [102]:
df["Geography"] = df["Geography"].progress_apply(lambda x: state_mapper[x])
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 332397/332397 [00:00<00:00, 1244404.95it/s]


In [103]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: "Yes" if x > 7 else "No")
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: "Yes" if x >= 3 else "No" )
df["HasCrCard"] = df["HasCrCard"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["IsActiveMember"] = df["IsActiveMember"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["Exited"] = df["Exited"].progress_apply(lambda x: "Exited" if x == 1 else "Not Exited")
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: 350 if x <=350 else x)

100%|██████████| 332397/332397 [00:00<00:00, 1070852.61it/s]


In [ ]:
def

df["Financial_year"] = df[["month", "year"]].progress_apply(financial_year, axis=1)

In [94]:
def cs_category(x):
    if x > 800:
        return "Exceptional"
    if x > 740:
        return "Very Good"
    if x > 670:
        return "Good"
    if x > 580:
        return "Fair"
    else:
        return "poor"
df["CS_Category"] = df["CreditScore"].progress_apply(cs_category)

df["SS_Category"] = df["Satisfaction Score"].progress_apply(lambda x: "Poor" if x <3 else "Good" if x < 4 else "Very Good")

df["Quarter"] = df["month"].progress_apply(lambda x: "Q4" if int(x)<=3 else "Q1" if int(x)<=6 else "Q2" if int(x)<=9 else "Q3" )

month_dic = {'May':5, 'Oct':10, 'Nov':11, 'Feb':2, 'Apr':4, 'Jul':7, 'Jan':1, 'Sep':9, 'Jun':6,
       'Aug':8, 'Dec':12, 'Mar':3}
month_dic = {v:k for k,v in month_dic.items()}

df["month"] = df["month"].progress_apply(lambda x: month_dic[int(x)])

100%|██████████| 332397/332397 [00:00<00:00, 1138933.40it/s]


In [95]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,date,City,month,year,Account_type,Other Accounts,Salary Account,CS_Category,SS_Category,Quarter
0,15658241,Olmstead,350.0,Louisiana,Female,81,48,438091.980941,2,Yes,...,1-2-2023,Kenner,Feb,2023,Gold,Yes,No,poor,Poor,Q4
1,15627162,Blesing,615.0,Louisiana,Male,27,6,155245.230616,1,Yes,...,1-3-2020,Shreveport,Mar,2020,Normal,No,No,Fair,Poor,Q4
2,15986692,Leiva,665.0,Louisiana,Male,35,7,157859.682813,2,Yes,...,1-10-2020,Lafayette,Oct,2020,Normal,No,Yes,Fair,Very Good,Q3
3,15443069,Jamieson,541.0,Maine,Male,36,7,0.000000,2,No,...,1-10-2020,Portland,Oct,2020,Normal,No,Yes,poor,Very Good,Q3
4,15611786,Tsui,621.0,Idaho,Female,69,9,0.000000,1,No,...,1-1-2020,Meridian,Jan,2020,Premium,Yes,Yes,Fair,Very Good,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332392,15308570,Daigle,450.0,Louisiana,Female,43,14,313145.164928,1,Yes,...,1-1-2022,Lafayette,Jan,2022,Premium,Yes,Yes,poor,Very Good,Q4
332393,15864602,Griffith,350.0,Maine,Male,52,33,684971.208280,2,Yes,...,1-7-2022,Portland,Jul,2022,Gold,Yes,No,poor,Poor,Q2
332394,15626485,Lu,595.0,Maine,Female,26,8,85203.608400,1,Yes,...,1-1-2020,Portland,Jan,2020,Premium,Yes,No,Fair,Poor,Q4
332395,15984022,Battle,350.0,Louisiana,Male,53,30,425846.088483,1,Yes,...,1-5-2022,Lafayette,May,2022,Gold,Yes,Yes,poor,Very Good,Q1


In [78]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Customer Status', 'Complain', 'Satisfaction Score',
       'Card Type', 'Point Earned', 'date', 'City', 'month', 'year',
       'Account_type', 'Other Accounts', 'Salary Account', 'CS_Category',
       'SS_Category', 'Quarter'],
      dtype='object')

In [96]:
df.rename(columns={'Exited': 'Customer Status'}, inplace=True)

In [79]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)